# Normalization to Control: This notebook provides a new normaliaztion to control method based on gem (made from experiment_ID and 10xrun) column

In [1]:
import sys
sys.path.append("..")
import Templates.tools.DatasetDB as DB
import Templates.tools.normalization_replogle as norm
import scanpy as sc
from tqdm import tqdm
import os
import time

In [2]:
#DSID = DatasetID
your_experiment = "raw_qc" # A string that is a key known for our desired experimnet that you wish to work on
Version = '1'

## Load the dataset

In [ ]:
if use_DSDB:
    ## Instantiate a dataset object given our dataset info
    DS = DB.DATASET(DSID, DEV)
    adata, adatas = DS.load_dataset(Version, your_experiment,Corr=False)
else:
    adata = sc.read_h5ad(raw_qc)

In [ ]:
adata

In [8]:
adata.obs

,Sample,Barcode,DemuxType_crispr,DemuxAssignment_crispr,DemuxType_hashing,DemuxAssignment_hashing,cellline,timepoint,HTO,NGS_ID,...,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,qc_pass,S_score,G2M_score,phase,SCN_class
SAM24443678_rep1-AAACCCAAGGAATCGC,SAM24443678,AAACCCAAGGAATCGC,singlet,ENSG00000135093_3,singlet,SAM24439045,DLD1,day5,HTO-7,NGS5570,...,4502,113,2.509996,1005,22.323412,True,-0.125123,0.574766,G2M,Untreatedclus_DLD1
SAM24443678_rep1-AAACCCAAGTCTTCCC,SAM24443678,AAACCCAAGTCTTCCC,singlet,ENSG00000181915_1,singlet,SAM24439044,DLD1,day5,HTO-5,NGS5570,...,17960,457,2.544543,3493,19.448775,True,-0.132656,0.891312,G2M,Untreatedclus_DLD1
SAM24443678_rep1-AAACCCACACAGCTGC,SAM24443678,AAACCCACACAGCTGC,singlet,ENSG00000288711_4,singlet,SAM24439044,DLD1,day5,HTO-4,NGS5570,...,10160,286,2.814961,2186,21.515748,True,0.000873,-0.068536,S,Untreatedclus_DLD1
SAM24443678_rep1-AAACCCACATGAATAG,SAM24443678,AAACCCACATGAATAG,singlet,ENSG00000103365_2,singlet,SAM24439045,DLD1,day5,HTO-18,NGS5570,...,10772,392,3.639064,2119,19.671370,True,0.134622,-0.171513,S,Untreatedclus_DLD1
SAM24443678_rep1-AAACCCATCGACCATA,SAM24443678,AAACCCATCGACCATA,singlet,ENSG00000139190_2,singlet,SAM24439045,DLD1,day5,HTO-7,NGS5570,...,9160,383,4.181223,1452,15.851528,True,0.075554,0.427137,G2M,Untreatedclus_DLD1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAM24443678_rep24-TTTGTTGGTATGAGGC,SAM24443678,TTTGTTGGTATGAGGC,singlet,ENSG00000185024_4,singlet,SAM24439045,DLD1,day5,HTO-8,NGS5570,...,5415,305,5.632502,993,18.337950,True,-0.594545,-0.008441,G1,Untreatedclus_DLD1
SAM24443678_rep24-TTTGTTGGTTTACGTG,SAM24443678,TTTGTTGGTTTACGTG,singlet,ENSG00000073282_4,singlet,SAM24439044,DLD1,day5,HTO-15,NGS5570,...,8771,126,1.436552,1507,17.181621,True,-0.381656,1.300689,G2M,Untreatedclus_DLD1
SAM24443678_rep24-TTTGTTGTCAGACCCG,SAM24443678,TTTGTTGTCAGACCCG,singlet,ENSG00000169946_1,singlet,SAM24439044,DLD1,day5,HTO-15,NGS5570,...,8692,333,3.831109,1544,17.763461,True,0.138666,-0.371232,S,Untreatedclus_DLD1
SAM24443678_rep24-TTTGTTGTCCTTATAC,SAM24443678,TTTGTTGTCCTTATAC,singlet,ENSG00000089775_3,singlet,SAM24439043,DLD1,day5,HTO-10,NGS5570,...,8220,468,5.693431,1311,15.948905,True,-0.163334,-0.185702,G1,Untreatedclus_DLD1


### correct gene_symbol for ENSGIDs

In [9]:
adata.obs['gene_symbol'] = adata.obs['gene_symbol'].apply(lambda x:x.split('_')[0])

### Create gem column from NGS_ID and 10xrun

In [10]:
adata.obs['gem'] = adata.obs['NGS_ID'].astype(str) + '-' + adata.obs['10Xrun'].astype(str)

### get control guides

In [11]:
idx = adata.obs.query("gene_symbol == 'NTC'").index

### Calculate median_umi_count

In [12]:
median_umi_count=None

In [13]:
if median_umi_count is None:
    median_umi_count = norm.check_matrix_and_densify(adata[idx,:].layers['counts']).sum(axis=1).median()
print('Normalizing all cells to {0} UMI...'.format(median_umi_count))

     Densifying matrix...
Normalizing all cells to 9151.0 UMI...


## Extract all the gems available in this anndata

In [14]:
gems = adata.obs['gem'].unique()

## make smaller adatas for each gems and store them

In [15]:
for gm in tqdm(gems):
    outfile = os.path.join(out_dir, f"{gm}.h5ad")
    if os.path.exists(outfile):
        continue
    else:
        t = time.time()
        small_adata = adata[adata.obs['gem']==gm].to_memory()
        gem_group_matrices = norm.proc_gem(small_adata,median_umi_count)
        gem_group_matrices.write(outfile)
        print(time.time() - t)

  0%|          | 0/24 [00:00<?, ?it/s]

     Densifying matrix...
     Densifying matrix...
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to median
     Scaling matrix to control
     Done.


  4%|▍         | 1/24 [00:42<16:13, 42.33s/it]

42.3271222114563
     Densifying matrix...
     Densifying matrix...
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to median
     Scaling matrix to control
     Done.


  8%|▊         | 2/24 [01:34<17:43, 48.34s/it]

...

49.94718527793884
     Densifying matrix...
     Densifying matrix...
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to median
     Scaling matrix to control
     Done.


 96%|█████████▌| 23/24 [18:23<00:49, 49.44s/it]

46.68066453933716
     Densifying matrix...
     Densifying matrix...
     Determining scale factors...
     Normalizing matrix to median
     Normalizing control matrix to median
     Scaling matrix to control
     Done.


100%|██████████| 24/24 [19:19<00:00, 48.32s/it]

56.50728940963745
